In [1]:
import unittest
from collections import defaultdict
import pdb
import datetime
from decimal import *
getcontext().prec = 6

In [2]:
init_price_token = [datetime.datetime(1971,1,1,0,0,0).timestamp(), 0]

In [126]:
class Fund:
    # ToDo - Replace token_name -> token_address

    curr_iteration = 0
    
    def __init__(self):
        self.token_balances = {'WETH': 0, 'DAI': 0}
        self.token_prices = {'WETH': None, 'DAI': None}
        self.shareholders_share = {}
    
    def tick(self):
        self.curr_iteration += 1
    
    def get_price(self, token_name):
        if token_name == 'WETH':
            return Decimal(3000) * (1 + (self.curr_iteration*Decimal('0.5'))) # 10% increase at each iteration
        elif token_name == 'DAI':
            return Decimal(1)
        else:
            raise Exception('price not found')
    
    def refresh_prices(self, token_name):
        self.token_prices[token_name] = self.get_price(token_name)
        # refresh other tokens
        for k,v in self.token_prices.items():
            if k != token_name:
                self.token_prices[k] = self.get_price(k)
        
    
    def emergency_price_update(self):
        # ToDo - Update all tokens
        pass

    # ToDo - Update prices whenever deposit() or withdraw()
    
    def deposit(self, depositor, token_name, token_quantity):
        
        self.refresh_prices(token_name)
        print('token_prices', self.token_prices)
        
        # get total value of fund
        old_usd_balance = Decimal(0)
        for temp_token_name, temp_token_balance in self.token_balances.items():
            temp_token_price = self.token_prices[temp_token_name]
            token_usd_value = temp_token_price * temp_token_balance
            #pdb.set_trace()
            old_usd_balance += token_usd_value
        # sum with token_quantity * price
        token_price = self.token_prices[token_name]
        new_deposit = token_quantity * token_price
        # determine pct of depositor
        new_usd_balance = (new_deposit + old_usd_balance)
        #pdb.set_trace()
        pct_depositor = new_deposit / new_usd_balance
        
        # updates
        self.shareholders_share[depositor] = pct_depositor
        self.token_balances[token_name] += token_quantity
        
        # determine pct of previous depositors
        for address, pct_total_holdings in self.shareholders_share.items():
            if not address == depositor:
                new_pct = (pct_total_holdings*old_usd_balance) / new_usd_balance
                self.shareholders_share[address] = new_pct
        
        print ('shareholders {} old_usd_balance {} new_usd_balance {} token balances {}'.format(
            self.shareholders_share, old_usd_balance, new_usd_balance, self.token_balances))

In [110]:
# test deposit
f = Fund()
initial_balance = f.token_balances['DAI']
unittest.TestCase().assertEqual(initial_balance, 0, 'oi')
deposit_alice = 100
f.deposit('alice', 'DAI', deposit_alice)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice, 'ops')
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal(1), 'ops')

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('1')} old_usd_balance 0.0 new_usd_balance 100.0
token balances {'WETH': 0, 'DAI': 100}


In [111]:
deposit_bob = 300
f.deposit('bob', 'DAI', deposit_bob)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice + deposit_bob, 'token balance does not match')
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal(0.25), 'token pct Alice does not match')
unittest.TestCase().assertEqual(f.shareholders_share['bob'], Decimal(0.75), 'token pct Alice does not match')

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('0.25'), 'bob': Decimal('0.75')} old_usd_balance 100.0 new_usd_balance 400.0
token balances {'WETH': 0, 'DAI': 400}


In [112]:
deposit_eve = 600
f.deposit('eve', 'DAI', deposit_eve)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice + deposit_bob + deposit_eve)
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal('0.1'))
unittest.TestCase().assertEqual(f.shareholders_share['bob'], Decimal('0.3'))
unittest.TestCase().assertEqual(f.shareholders_share['eve'], Decimal('0.6'))

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('0.10'), 'bob': Decimal('0.30'), 'eve': Decimal('0.6')} old_usd_balance 400.0 new_usd_balance 1000.0
token balances {'WETH': 0, 'DAI': 1000}


In [113]:
# Test WETH
fweth = Fund()
deposit_weth = 1
fweth.deposit('alice2','WETH',deposit_weth)
unittest.TestCase().assertEqual(fweth.token_balances['WETH'], deposit_weth)
unittest.TestCase().assertEqual(fweth.shareholders_share['alice2'], Decimal(1))
deposit_dai = 3000
fweth.deposit('bob2','DAI',deposit_dai)
#unittest.TestCase().assertEqual(fweth.token_balances['DAI'], deposit_dai)
#unittest.TestCase().assertEqual(fweth.shareholders_share['alice2'], Decimal('0.5'))
#unittest.TestCase().assertEqual(fweth.shareholders_share['bob2'], Decimal('0.5'))

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice2': Decimal('1')} old_usd_balance 0.0 new_usd_balance 3000.0
token balances {'WETH': 1, 'DAI': 0}
token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice2': Decimal('0.5'), 'bob2': Decimal('0.5')} old_usd_balance 3000.0 new_usd_balance 6000.0
token balances {'WETH': 1, 'DAI': 3000}


In [114]:
# Another test for when price changes

In [132]:
ftick = Fund()
deposit_john = 1
ftick.deposit('alice','WETH',deposit_john)
unittest.TestCase().assertEqual(ftick.token_prices['WETH'], Decimal(3000))
# now weth worth 4500
ftick.tick()
ftick.deposit('bob','WETH',deposit_john)
unittest.TestCase().assertEqual(ftick.shareholders_share['bob'], Decimal('0.5'))
unittest.TestCase().assertEqual(ftick.shareholders_share['alice'], Decimal('0.5'))

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('1')} old_usd_balance 0.0 new_usd_balance 3000.0 token balances {'WETH': 1, 'DAI': 0}
token_prices {'WETH': Decimal('4500.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('0.5'), 'bob': Decimal('0.5')} old_usd_balance 4500.0 new_usd_balance 9000.0 token balances {'WETH': 2, 'DAI': 0}


In [133]:
ftickDAI = Fund()
deposit_alice = 1
ftickDAI.deposit('alice','WETH',deposit_alice)
ftickDAI.tick()
deposit_bob = 4500
ftickDAI.deposit('bob','DAI',deposit_bob)
unittest.TestCase().assertEqual(ftickDAI.shareholders_share['bob'], Decimal('0.5'))
unittest.TestCase().assertEqual(ftickDAI.shareholders_share['alice'], Decimal('0.5'))

token_prices {'WETH': Decimal('3000.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('1')} old_usd_balance 0.0 new_usd_balance 3000.0 token balances {'WETH': 1, 'DAI': 0}
token_prices {'WETH': Decimal('4500.0'), 'DAI': Decimal('1')}
shareholders {'alice': Decimal('0.5'), 'bob': Decimal('0.5')} old_usd_balance 4500.0 new_usd_balance 9000.0 token balances {'WETH': 1, 'DAI': 4500}
